<a href="https://colab.research.google.com/github/ItaSsa/GettingSomeAzureDevOpsOrganizations/blob/main/Proj01_Notebook_ToCSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Let's go!:)

# Adcquiring this pretty libary :)

!pip install azure-devops

In [ ]:


# Now let's importe the libraries 
# We'll use WIQL 

from azure.devops.connection import Connection
from msrest.authentication import BasicAuthentication
from azure.devops.v5_1.work_item_tracking.models import Wiql
import pandas as pd

In [ ]:
# Setting connection variables
token_list = (<token_org1>,<token_org2>)
organization_name = (<Org_name1>,<Org_name2>)
ad_link ='https://dev.azure.com/'


In [ ]:
#  Now putting my personal touch in a code whose source is bellow :)
# This code had been adapted from this: https://stackoverflow.com/questions/52810441/how-to-extract-workitems-from-query-in-vsts-azure-devops-with-python-rest-api
 
df = pd.DataFrame([])

for i in range(len(organization_name)):
      token = token_list[i]
      org_name = organization_name[i]
      team_instance = ad_link + org_name

      #Get a connection
      credentials = BasicAuthentication("", token)
      connection = Connection(base_url=team_instance, creds=credentials)

      #Returning workitens to dataframe
      #Functions
      def print_work_itemsI(work_items):
        workItemType_list = []
        title_list = []
        state_list =[]
        asseginedTo_list =[]
        areaPath_list  =[]

        for work_item in work_items:
              workItemType_list.append(work_item.fields["System.WorkItemType"])
              title_list.append(work_item.fields["System.Title"])
              state_list.append(work_item.fields["System.State"])
              user_t = list(work_item.fields["System.AssignedTo"].values())
              asseginedTo_list.append(user_t[0])
              areaPath_list.append(work_item.fields["System.AreaPath"])

        # Here we are mounting the final result 
        data = [workItemType_list,title_list, state_list,asseginedTo_list,areaPath_list]
        data_frame = pd.DataFrame(data)
        data_frame = data_frame.T
        data_frame.columns = ['WorkItemType','Title','State','AssignedTo','AreaPath']
        return data_frame
      
      # Get a client (the "core" client provides access to projects, teams, etc)
      # Source https://github.com/microsoft/azure-devops-python-api
      wit_client = connection.clients.get_work_item_tracking_client()


      def get_TC_from_query(query):
          query_wiql = Wiql(query=query)
          results = wit_client.query_by_wiql(query_wiql).work_items
          # WIQL query gives a WorkItemReference => we get the corresponding WorkItem from id
          work_items = (wit_client.get_work_item(int(result.id)) for result in results)
          return print_work_itemsI(work_items)

      #Mounting queries
      #Filters
      workitem_type = """'issue'"""

      #The query :)
      query_A = f"""\
      SELECT
              [System.Id],
              [System.WorkItemType],
              [System.Title],
              [System.State],
              [System.AreaPath],
              [System.AssignedTo]
      FROM workitems
      WHERE
            [System.WorkItemType] = {workitem_type}
              
      """
      #Exporting Data
      df = df.append(get_TC_from_query(query_A),ignore_index = True)

# Getting the final result
# Here we could put this data in a database for example
df.to_csv('WorkItemTable.csv', encoding='utf-8',sep=';')
#print(df)
